## This notebook visualises a two finger gripper with a simple bottle cap

In [1]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration
from pydrake.all import RobotDiagramBuilder

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [63]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

gripper = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]
obstacle1 = parser.AddModels(file_name="my_sdfs/obstacle.sdf")[0]
obstacle2 = parser.AddModels(file_name="my_sdfs/obstacle.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

# Weld the obstacle to the world frame (adjust pose as needed)
obstacle_pose1 = RigidTransform(RotationMatrix(), [0.01, 0.035, 0.02])  # Adjust position
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("obstacle_link", obstacle1),
    obstacle_pose1)

obstacle_pose2 = RigidTransform(RotationMatrix(), [-0.035, -0.005, 0.02])  # Adjust position
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("obstacle_link", obstacle2),
    obstacle_pose2)

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))

# Fix right finger to left finger
right_finger_joint = plant.GetJointByName("right_finger_sliding_joint", gripper)
left_finger_joint = plant.GetJointByName("left_finger_sliding_joint", gripper)

# Set default joint translation to 0.025
right_finger_joint.set_default_translation(0.025)
left_finger_joint.set_default_translation(-0.025)

plant.Finalize()

In [64]:
B_cap = plant.GetBodyByName("cap_link", cap)
B_gripper = plant.GetBodyByName("body", gripper)
B_finger = plant.GetBodyByName("left_finger", gripper)


# AddFrameTriadIllustration(
#     plant=plant,
#     scene_graph=scene_graph,
#     body=B_cap,
#     length=0.06
# )

# AddFrameTriadIllustration(
#     plant=plant,
#     scene_graph=scene_graph,
#     body=B_gripper,
#     length=0.10
# )



In [65]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

In [66]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [67]:
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

p_GgraspO = [0, .115, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2).multiply(RotationMatrix.MakeZRotation(np.pi / 4))

X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

plant.SetFreeBodyPose(plant_context, B_gripper, X_WGgrasp)

plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.033
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.033
)

<PrismaticJoint name='right_finger_sliding_joint' index=1 model_instance=2>

In [62]:
diagram.ForcedPublish(context)